In [28]:
spark.stop() 

# Set paths

DEVEL_SRC must contain the directory use-cases and pykhaos

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when '-' is removed from name, remove also this line and adapt imports 
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
# AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
# if AMDOCS_SRC not in sys.path: 
#     sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
        
from project.project_generic import Project

import pykhaos.utils.notebooks as nb

import time
start_time = time.time()

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    EXTERNAL_LIB = os.path.join(os.environ.get('BDA_USER_HOME', ''), "lib", "external_libs")
    if EXTERNAL_LIB not in sys.path:
        sys.path.append(EXTERNAL_LIB)
    # feel free from commenting this line and the other ones that begin with "%%notify" if you do not have 
    # the extension installed or copy de lib from /var/SP/data/home/csanc109/lib/external_libs/jupyternotify/
    %load_ext jupyternotify 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

20190830-115714 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20190830_115714.log


<IPython.core.display.Javascript object>

In [2]:
from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, asc, to_date, create_map, sum as sql_sum
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
from churn.datapreparation.general.data_loader import get_unlabeled_car, get_port_requests_table, get_numclients_under_analysis
from churn.utils.constants import PORT_TABLE_NAME
from churn.utils.udf_manager import Funct_to_UDF
from pyspark.sql.functions import substring, datediff, row_number
from pykhaos.utils.date_functions import move_date_n_days, move_date_n_cycles
from pykhaos.utils.hdfs_functions import check_hdfs_exists
from pykhaos.modeling.model_performance import get_lift

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Start spark context

In [3]:
from churn.utils.general_functions import init_spark
spark = init_spark("billing")
sc = spark.sparkContext

Ended spark session: 39.6994678974 secs | default parallelism=2


## Write here the prediction date

In [4]:
closing_day = "20190521"


## Read output of both models and compute simple stats

In [12]:
from pyspark.sql.functions import mean as sql_avg, greatest


partition_date = "year={}/month={}/day={}".format(int(closing_day[:4]), int(closing_day[4:6]), int(closing_day[6:]))

# model1 is Cristina's model
df_model1 = (spark.read.load("/data/udf/vf_es/churn/triggers/model1_50k/" + partition_date)
                            .select(['NIF_CLIENTE', 'calib_model_score', 'label']).withColumnRenamed("calib_model_score", "calib_model_score_model1")                                                                         
                 .withColumnRenamed("label", "label_model1")).where(col("NIF_CLIENTE").isNotNull()).drop_duplicates(["NIF_CLIENTE"])


# model2 is Alvaro's model
df_model2 = (spark.read.load("/data/udf/vf_es/churn/triggers/model2_50k/" +  partition_date).select(['NIF_CLIENTE', 'calib_model_score', "label"])
                                                                         .withColumnRenamed("calib_model_score", "calib_model_score_model2")
                                                                         .withColumnRenamed("label", "label_model2")).where(col("NIF_CLIENTE").isNotNull()).drop_duplicates(["NIF_CLIENTE"])

          
print("STATS") # just a check, count must be 50k in both cases
print('\tNIFs model1 = {}'.format(df_model1.count()))
print('\tNIFs model2 = {}'.format(df_model2.count()))

# compute common nifs
df_inner = df_model1.join(df_model2, on=["nif_cliente"], how="inner")
common_nifs = df_inner.count()
print("\t Common nifs = {}".format(common_nifs))

# NIFs in Cris' model not included in Alvaro's model
df_model1_no_model2 = df_model2.select("nif_cliente").join(df_model1.select("nif_cliente"), ['nif_cliente'], 'right').where(df_model2['nif_cliente'].isNull())

# NIFs in Alvaro's model not included in Cris' model
df_model2_no_model1 = df_model1.select("nif_cliente").join(df_model2.select("nif_cliente"), ['nif_cliente'], 'right').where(df_model1['nif_cliente'].isNull())

# Absurd print, since new NIFs are 50k - common, but good for checking
print("NIFs nuevos model1 sobre model2 = {} [check={}]".format(df_model1_no_model2.count(), df_model1_no_model2.count() + common_nifs))
print("NIFs nuevos model2 sobre model1 = {} [check={}]".format(df_model2_no_model1.count(), df_model2_no_model1.count() + common_nifs))

from churn.analysis.triggers.trigger_billing_ccc.trigger_smart_exploration import get_customer_master

_, volume_ref, churn_ref = get_customer_master(spark, closing_day)

print("Reference base: churn_rate={:.2f} volume={}".format(100.0*churn_ref, volume_ref))
                                                                                                               
   

STATS
	NIFs model2 = 50000
	NIFs model1 = 50000
	 Common nifs = 15699
NIFs nuevos model1 sobre model2 = 34301 [check=50000]
NIFs nuevos model2 sobre model1 = 34301 [check=50000]
Reference base: churn_rate=3.25 volume=3899811


### Join both dataframes - if NIF is in both, keep the highest

In [13]:

pos_max_udf = udf(lambda milist: sorted([(vv, idx) for idx, vv in enumerate(milist)], key=lambda tup: tup[0], reverse=True)[0][1], IntegerType())

df_all = df_model2.join(df_model1, on=["nif_cliente"], how="full").fillna(0)  

df_all = df_all.withColumn("calib_model_score", greatest(*["calib_model_score_model1", "calib_model_score_model2"]))

df_all = df_all.withColumn("score_array", array(["calib_model_score_model1", "calib_model_score_model2"]))  
df_all = df_all.withColumn("pos_max", pos_max_udf(col("score_array")))  


df_all = df_all.withColumn("label", greatest(*["label_model1", "label_model2"])).where(col("NIF_CLIENTE").isNotNull()).drop_duplicates(["NIF_CLIENTE"])
df_all = df_all.withColumn("model", when(col("pos_max")==0, "csanc109").when(col("pos_max")==1, "asaezco").otherwise("unknown"))

df_all = df_all.drop("pos_max", "score_array")
df_all = df_all.sort(desc("calib_model_score"))

df_all = df_all.cache()

num_rows = df_all.count()

print("Combination of both models has {} unique nifs".format(num_rows))


Combination of both models has 84301 unique nifs


In [14]:
df_all.columns

['NIF_CLIENTE',
 'calib_model_score_model2',
 'label_model2',
 'calib_model_score_model1',
 'label_model1',
 'calib_model_score',
 'label',
 'model']

### Write combination 

In [86]:
path_to_save = "/data/udf/vf_es/churn/triggers/model_combined/"

df_all = df_all.withColumn("day", lit(int(closing_day[6:])))
df_all = df_all.withColumn("month", lit(int(closing_day[4:6])))
df_all = df_all.withColumn("year", lit(int(closing_day[:4])))

start_time = time.time()
print("Started saving - {} for closing_day={}".format(path_to_save, closing_day))
(df_all.coalesce(1).write.partitionBy('year', 'month', 'day').mode("append").format("parquet").save(path_to_save))
print("Elapsed time saving {} minutes".format((time.time()-start_time)/60.0))

Started saving - /user/csanc109/projects/triggers/trigger_exploration_combined/ for closing_day=20190521
Elapsed time saving 0.00830854972204 minutes


### Show the volume vs lift for model combined

In [17]:

myschema = df_all.schema

for ii in [num_rows, 50000, 40000, 30000, 20000, 15000, 12000, 10000, 5000, 3000, 2000]: 
    
    start_time_ii = time.time()
    df_top = spark.createDataFrame(df_all.head(ii),  schema=myschema)
    churn_rate_top =  df_top.select(sql_avg('label').alias('churn_rate')).rdd.first()['churn_rate']
    print("TOP{} - LIFT={:.2f} churn_rate={:.2f}% churn_rate_ref={:.2f}%".format(ii, 
                                                                                          churn_rate_top/churn_ref,
                                                                                          churn_rate_top*100.0,
                                                                                          churn_ref*100.0, 
                                                                                          (time.time()-start_time_ii)/60.0))
    inner_nifs = df_top.join(df_inner, on=["NIF_CLIENTE"], how="inner").select("NIF_CLIENTE").count()
    from_model1_nifs = df_top.join(df_model1_no_model2, on=["NIF_CLIENTE"], how="inner").select("NIF_CLIENTE").count()
    from_model2_nifs = df_top.join(df_model2_no_model1, on=["NIF_CLIENTE"], how="inner").select("NIF_CLIENTE").count()
    
    print("\t COMMON={} [{:.2f}%] || FROM_MODEL1={} [{:.2f}%] || FROM_MODEL2={} [{:.2f}%]".format(inner_nifs, 100.0*inner_nifs/ii, from_model1_nifs, 100.0*from_model1_nifs/ii, from_model2_nifs, 100.0*from_model2_nifs/ii))
                                                           
                                                           


TOP84301 - LIFT=5.70 churn_rate=18.50% churn_rate_ref=3.25%
	 COMMON=15699 [18.62%] || FROM_MODEL1=34301 [40.69%] || FROM_MODEL2=34301 [40.69%]
TOP50000 - LIFT=7.21 churn_rate=23.41% churn_rate_ref=3.25%
	 COMMON=13538 [27.08%] || FROM_MODEL1=13269 [26.54%] || FROM_MODEL2=23193 [46.39%]
TOP40000 - LIFT=8.00 churn_rate=25.98% churn_rate_ref=3.25%
	 COMMON=11748 [29.37%] || FROM_MODEL1=10642 [26.61%] || FROM_MODEL2=17610 [44.02%]
TOP30000 - LIFT=9.14 churn_rate=29.67% churn_rate_ref=3.25%
	 COMMON=9329 [31.10%] || FROM_MODEL1=8155 [27.18%] || FROM_MODEL2=12516 [41.72%]
TOP20000 - LIFT=10.95 churn_rate=35.52% churn_rate_ref=3.25%
	 COMMON=6534 [32.67%] || FROM_MODEL1=5349 [26.75%] || FROM_MODEL2=8117 [40.59%]
TOP15000 - LIFT=12.33 churn_rate=40.00% churn_rate_ref=3.25%
	 COMMON=5101 [34.01%] || FROM_MODEL1=3921 [26.14%] || FROM_MODEL2=5978 [39.85%]
TOP12000 - LIFT=13.44 churn_rate=43.61% churn_rate_ref=3.25%
	 COMMON=4273 [35.61%] || FROM_MODEL1=2978 [24.82%] || FROM_MODEL2=4749 [39.58%]


# THE END - the goal of the script ends here!

### This block is just for me, check de volume vs lift for model 1

In [19]:

df_model1 = (spark.read.load("/data/udf/vf_es/churn/triggers/model1_50k/" + partition_date)
                            .select(['NIF_CLIENTE', 'model_score', 'label']))
myschema = df_model1.schema

df_model1 = df_model1.sort(desc("model_score"))


for ii in [50000, 40000, 30000, 20000, 16000, 12000, 10000, 5000, 3000, 2000, 1000, 2500, 8000, 15000, 25000, 35000, 45000, num_rows]:
    
    start_time_ii = time.time()
    df_top = spark.createDataFrame(df_model1.head(ii),  schema=myschema)
    churn_rate_top =  df_top.select(sql_avg('label').alias('churn_rate')).rdd.first()['churn_rate']
    print("TOP{} - LIFT={:.2f} churn_rate={:.2f}% churn_rate_ref={:.2f}%".format(ii, 
                                                                                          churn_rate_top/churn_ref,
                                                                                          churn_rate_top*100.0,
                                                                                          churn_ref*100.0, 
                                                                                          (time.time()-start_time_ii)/60.0))
                

TOP50000 - LIFT=6.44 churn_rate=20.91% churn_rate_ref=3.25%
TOP40000 - LIFT=6.91 churn_rate=22.42% churn_rate_ref=3.25%
TOP30000 - LIFT=7.72 churn_rate=25.06% churn_rate_ref=3.25%
TOP20000 - LIFT=9.05 churn_rate=29.36% churn_rate_ref=3.25%
TOP16000 - LIFT=9.62 churn_rate=31.23% churn_rate_ref=3.25%
TOP12000 - LIFT=10.77 churn_rate=34.96% churn_rate_ref=3.25%
TOP10000 - LIFT=11.54 churn_rate=37.44% churn_rate_ref=3.25%
TOP5000 - LIFT=14.75 churn_rate=47.88% churn_rate_ref=3.25%
TOP3000 - LIFT=16.94 churn_rate=54.97% churn_rate_ref=3.25%
TOP2000 - LIFT=18.07 churn_rate=58.65% churn_rate_ref=3.25%
TOP1000 - LIFT=19.17 churn_rate=62.20% churn_rate_ref=3.25%
TOP2500 - LIFT=17.51 churn_rate=56.84% churn_rate_ref=3.25%
TOP8000 - LIFT=12.65 churn_rate=41.05% churn_rate_ref=3.25%
TOP15000 - LIFT=9.83 churn_rate=31.90% churn_rate_ref=3.25%
TOP25000 - LIFT=8.34 churn_rate=27.08% churn_rate_ref=3.25%
TOP35000 - LIFT=7.26 churn_rate=23.56% churn_rate_ref=3.25%
TOP45000 - LIFT=6.66 churn_rate=21.60%

In [ ]:
# Check incrementals

In [ ]:
#incremental - not finished

# df_C = df_cris.select("nif_cliente").join(df_cris2.select("nif_cliente"), ['nif_cliente'], 'right').where(df_cris['nif_cliente'].isNull())
# print("RESUMEN - INCREMENTALES", df_C.count())